In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

    model_A6_001, input_features_A6_001 = pre_models.Input_A6_001('weights/')
    
    outputs = ['Input_A6_001']

    preprocessings = ['skewness']

    models = [model_A6_001]

    input_features = [input_features_A6_001]

    result = {}

    for i in range(len(outputs)):

            if preprocessings[i] == 'skewness':
                X = skewness_pure(X_df, input_features[i])
            elif preprocessings[i] == 'min_max_normalization':
                X = min_max_normalization_pure(X_df, input_features[i])

            result[outputs[i]] = predict_mul(models[i], X)
    
    result_df = pd.DataFrame(result, columns = outputs)
    return result_df


In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=3
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A6_001.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
nn=data.drop(['Predict'], axis=1)
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A6_002', 'Input_A6_003', 'Input_A6_005'], dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [8]:
nn_ave(nn, y)

test_rmse_ave: 0.022347963558770896
[0.03333325832908795, 0.017020278284123804, 0.01633242524599913, 0.030278268267058935, 0.030563739557349385, 0.018769366297483193, 0.01926907045225762, 0.024975568057758888, 0.013471894118890776, 0.019465766977699264]


test_r2_ave: 0.41966863176291713
[-0.7015939314555524, 0.7676230551469987, 0.6839859382899027, -0.2203308209677044, 0.5793737733457265, 0.5917179174728999, 0.7444330341961635, 0.2143639380393384, 0.8454105727915103, 0.691702840769888]


## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,3)

train_rmse: 0.015629789219379738
test_rmse: 0.014470729413368688
test_r2: 0.801804072673548


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,2e-06)

test_rmse_ave: 0.014430997502281922
[0.008943306235656376, 0.013071210871812022, 0.012939423123554377, 0.01525210107549433, 0.020713636903298038, 0.010434264319936397, 0.01719925643356397, 0.016067519923427944, 0.014861325280574784, 0.014827930855501003]


test_r2_ave: 0.801730312273083
[0.8775111390545136, 0.8629461100820008, 0.8016487786139526, 0.6903473048690911, 0.8068049095825084, 0.8738216716410457, 0.7963883932699282, 0.6748465588485301, 0.8118788840826303, 0.8211093726866296]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.03],0.01)

test_rmse_ave: 0.014332522319325325
[0.007852690158250115, 0.012982532499464287, 0.012968389412896105, 0.015388599078798557, 0.021402535654027773, 0.009233184815251004, 0.01742243038328616, 0.016029697489503218, 0.015075677709783328, 0.014969485991992682]


test_r2_ave: 0.804237832582125
[0.9055640582150133, 0.8647994179465145, 0.8007597235496899, 0.684780057759734, 0.7937405639658286, 0.9011983588032993, 0.7910700678115694, 0.6763755589604344, 0.8064130153613525, 0.817677503447815]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,3,model)

train_rmse: 0.01679281095760865
test_rmse: 0.014703087024751349
test_r2: 0.790881695887998


## XGB

In [8]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 130, 
              max_depth = 1, 
              min_child_weight = 1,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.85,
              reg_alpha =  0,
              reg_lambda = 1)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.002472808015534328
[0.0004424669656394691, 0.0021229864794565486, 0.0014166562724227906, 0.001596696422350009, 0.0059347907807062086, 0.003341715799536615, 0.002670599807987617, 0.0022453110899450833, 0.0019153977173800795, 0.003041458819918865]


test_r2_ave: 0.9939601959677267
[0.9997001789671075, 0.9963846180869226, 0.9976224260845834, 0.9966064033741826, 0.9841403282553274, 0.9870580378486575, 0.9950909120315392, 0.9936504389502406, 0.9968750692520241, 0.9924735468266829]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 3,
            min_samples_split = 5,
            max_leaf_nodes = 5,
            min_samples_leaf =2,
            random_state = 42)  
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.004181618825684509
[0.002020761597096426, 0.0034743966616410924, 0.004331097545734795, 0.004048422384113505, 0.00642353672115828, 0.0032072181936474444, 0.008021583840430606, 0.005468102583731626, 0.001972294595174342, 0.002848774134116979]


test_r2_ave: 0.9817658418847554
[0.9937463934809763, 0.9903168084609834, 0.9777770873145517, 0.9781833959398453, 0.9814205955064246, 0.9880788510278739, 0.9557102282916768, 0.962341417621272, 0.9966866597562453, 0.9933969814477038]


In [9]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=2e-06)
elastic_mod=ElasticNet(alpha=[0.03], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 130, 
              max_depth = 1, 
              min_child_weight = 1,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.85,
              reg_alpha =  0,
              reg_lambda = 1)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 3,
            min_samples_split = 5,
            max_leaf_nodes = 5,
            min_samples_leaf =2,
            random_state = 42)  
vote_mod = VotingRegressor([ 
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])
# ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.0031267342409505004
[0.0008162828100104276, 0.0027088879769106363, 0.0025330556933649937, 0.0028849345160901845, 0.005866773924691412, 0.0030996833471337738, 0.004833415870824675, 0.003614811854135091, 0.0020160471199286156, 0.002893449296415192]


test_r2_ave: 0.9904968477858441
[0.9989795735820101, 0.9941137078181541, 0.9923985835415307, 0.9889213348923415, 0.9845017710011071, 0.9888648579524025, 0.9839197974401009, 0.9835425685053596, 0.9965380258312565, 0.9931882572941791]


In [10]:

stack_mod = StackingRegressor(regressors=[elastic_mod,random_mod,lr, vote_mod], 
                           meta_regressor=xgb_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.0023645811028507715
[0.0005801595433627931, 0.002678236315614131, 0.0005451093876617346, 0.001820930670888534, 0.0052189668412339, 0.0028572782021034702, 0.002375741638556085, 0.002461823644565344, 0.0022867153773822777, 0.0028208494071394422]


test_r2_ave: 0.9944792531084723
[0.9994845396395802, 0.9942461635443672, 0.9996479759197957, 0.9955863048143992, 0.9877354267135475, 0.9905383657169686, 0.9961150842359734, 0.9923668369826473, 0.9955460361195664, 0.9935257973978782]


In [11]:
ave(skew_data[title],y,0.4,stack_mod,0.2,vote_mod,0.4,xgb_mod)

test_rmse_ave: 0.0023575204591719536
[0.0004648412513245321, 0.002498266275756984, 0.0010552334372833693, 0.0020393176034966356, 0.005437217775356968, 0.0030064665058132205, 0.0026988381642701923, 0.001210923887343287, 0.002071167569902347, 0.0030929321211720035]


test_r2_ave: 0.9945722738433507
[0.9996690901344633, 0.9949934659374456, 0.9986808258322845, 0.9944641380526705, 0.9866881987879524, 0.9895245221930402, 0.9949865478597937, 0.99815318091798, 0.9963461312677359, 0.9922166374501411]


In [12]:
name='Input_A6_001'
test_pred=[]
seed=[2,3,6,7,1,8]
stack_w=0.4
vote_w=0.2
model_w=0.4
model=xgb_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)


final_test = pd.DataFrame(np.mean(test_pred, axis=0), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))
